In [8]:
import copy
import os
filepath = os.path.dirname( os.path.realpath( '__file__'))
datadir = os.path.realpath(os.path.join(filepath, ''))
os.chdir(datadir)

In [11]:
from nipype.interfaces.ants import Registration

reg = Registration()
reg.inputs.fixed_image = 'lhtemplate0.nii.gz'
#copy the inputs from the other script I was working on
reg.cmdline

ValueError: Registration requires a value for input 'shrink_factors'. For a list of required inputs, see Registration.help()